In [1]:
# Setup for Google Colab directories
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
%cd MLexercise_CarDimensions

# import glob
# import re
import pandas as pd
# file_paths = glob.glob("data/2*_en.csv")
dataframes = pd.read_csv("data/cleaned_data.csv")
print(dataframes.columns.tolist())
dataframes

/content/drive/MyDrive/Colab Notebooks/MLexercise_CarDimensions
['MAKE', 'MODEL', 'Model Year', 'Overall Length', 'Overall Width', 'Overall Height', 'Wheelbase', 'Curb Weight', 'Front End Length', 'Rear End Length', 'Side Glass Height', 'Body Side Height', 'Roof Width', 'Front Overhang', 'Rear Overhang', 'Track Width Front', 'Track Width Rear', 'Weight Distribution', 'Front Weight %', 'Rear Weight %']


,MAKE,MODEL,Model Year,Overall Length,Overall Width,Overall Height,Wheelbase,Curb Weight,Front End Length,Rear End Length,Side Glass Height,Body Side Height,Roof Width,Front Overhang,Rear Overhang,Track Width Front,Track Width Rear,Weight Distribution,Front Weight %,Rear Weight %
0,FIAT,124SPIDER2DRCONVERTIBLE,16,407.0,173.0,124.0,231.0,1114.0,155.0,79.0,31.0,67.0,116.0,85.0,91.0,152.0,150.0,54/46,54.0,46.0
1,RAMTRUCKS,1500CREWCAB57FTBOX2WD,18,592.0,208.0,197.0,367.0,2255.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
2,RAMTRUCKS,1500CREWCAB57FTBOX2WD,18,592.0,202.0,197.0,367.0,2255.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
3,RAMTRUCKS,1500CREWCAB57FTBOX4X4,18,592.0,208.0,197.0,367.0,2327.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
4,RAMTRUCKS,1500CREWCAB57FTBOX4X4,18,592.0,202.0,197.0,367.0,2327.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,BMW,Z4SDRIVE35I35IS2DRCONVERTIBLERWD,9,424.0,179.0,129.0,250.0,1610.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3760,BMW,Z4SDRIVE35I35IS2DRCONVRWD,9,424.0,179.0,129.0,250.0,1585.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3761,BMW,Z4SDRIVE35I35IS2DRCONVRWD,9,424.0,179.0,129.0,250.0,1590.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3762,ACURA,ZDX4DRSUVAWDTECHELITE,10,488.0,199.0,160.0,275.0,2064.0,133.0,109.0,33.0,88.0,120.0,109.0,104.0,172.0,171.0,58/42,58.0,42.0


In [2]:
# # Features and target variable
features = [
'Overall Length', 'Overall Width', 'Overall Height',
'Front End Length', 'Rear End Length',
'Side Glass Height', 'Body Side Height',
'Wheelbase', 'Front Overhang', 'Rear Overhang', # Sum = Overall Length
'Roof Width', 'Track Width Front', 'Track Width Rear', # ~ Overall Width
]

target = 'Curb Weight'

X = dataframes[features]
y = dataframes[target]

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

X_train_t = torch.tensor(X.values, dtype=torch.float32)
y_train_t = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

dataset = TensorDataset(X_train_t, y_train_t)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(X.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

model = Regressor()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

for epoch in range(100):
    for xb, yb in loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



In [4]:
# Convert the full DataFrame X into a torch tensor
X_tensor = torch.tensor(X.values, dtype=torch.float32)

# Disable gradient tracking (since we're only predicting)
with torch.no_grad():
    y_pred = model(X_tensor)

# # y_pred is a tensor -> convert back to NumPy or Pandas if needed
# y_pred_np = y_pred.numpy().flatten()  # 1D array


In [6]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

print("R² Score:", r2_score(y, y_pred))
print("MSE:", mean_squared_error(y, y_pred))
print("Mean target weight:", y.mean())
print("Mean target predicted:", y_pred.mean())
mape = mean_absolute_percentage_error(y, y_pred)
print("MAPE:", mape * 100, "%")


R² Score: 0.7093562807481305
MSE: 46559.3619280792
Mean target weight: 1808.4378320935175
Mean target predicted: 1832.1527
MAPE: 9.482365377066785 %


In [7]:
from notebooks.utils import compare_predictions

comparison = compare_predictions(y, y_pred)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(comparison.head(100))

    Actual    Predicted       Error    Error %
0   1114.0  1612.702881  498.702881  44.766865
1   2255.0  2463.994629  208.994629   9.268054
2   2255.0  2456.558594  201.558594   8.938297
3   2327.0  2463.994629  136.994629   5.887178
4   2327.0  2456.558594  129.558594   5.567623
5   2277.0  2510.714355  233.714355  10.264135
6   2277.0  2503.278320  226.278320   9.937563
7   2373.0  2510.714355  137.714355   5.803386
8   2373.0  2503.278320  130.278320   5.490026
9   2231.0  2306.809326   75.809326   3.397998
10  2198.0  2306.809326  108.809326   4.950379
11  2367.0  2306.809326  -60.190674   2.542910
12  2494.0  2306.809326 -187.190674   7.505640
13  2358.0  2306.809326  -51.190674   2.170936
14  2367.0  2306.809326  -60.190674   2.542910
15  2177.0  2443.681152  266.681152  12.249938
16  2177.0  2436.245361  259.245361  11.908377
17  2308.0  2443.681152  135.681152   5.878733
18  2308.0  2436.245361  128.245361   5.556558
19  2231.0  2306.809326   75.809326   3.397998
20  2198.0  2

In [9]:
comparison.to_csv("results/weight_prediction_PyTorch.csv", index=False)